## Import Libraries

In [ ]:
%tensorflow_version 1.x

import scipy.io
import scipy.signal
import random
import os
from os import environ
import numpy as np
import pandas as pd
import csv
import cv2
import h5py
from skimage import color
from google.colab import drive
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.initializers import random_uniform
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, LSTM, ConvLSTM2D,Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalAveragePooling2D, TimeDistributed, Reshape, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers
from tensorflow.initializers import random_uniform
from PIL import Image
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.utils.multiclass import unique_labels

print("Libraries imported")

TensorFlow 1.x selected.
Libraries imported


Using TensorFlow backend.


## User Defined Functions

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        pass
        #print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

def encode_label(target):
    res=[]
    if target=="N":
        res=0
    elif target=="A":
        res=1
    elif target=="R":
        res=2   
    elif target=="O":
        res=3
    mat = tf.keras.utils.to_categorical(res, num_classes=4)
    return mat

## Specify data location & Training parameters

In [ ]:
drive.mount('/content/drive', force_remount=True)
train_dir = '/content/drive/My Drive/Neural Network Project/TrainingSet4/'
batch_size=20
num_classes=4
epochs=50
save_dir='/content/drive/My Drive/Neural Network Project/'
model_name="FK_model.h5"

print("Drive mounted")

Mounted at /content/drive
Drive mounted


## Import the transformed dataset

In [ ]:
x_train=[]
y_train=[]

print("Loading Dataset...")

for target in os.listdir(train_dir):
    for file in os.listdir(train_dir+'/'+target):
        label=encode_label(target)
        y_train.append(label)
        image = Image.open(train_dir + '/' + target + '/' + file).convert("RGB")
        image = image.resize((300, 200), Image.ANTIALIAS)
        x = img_to_array(image)
        x_train.append(x)


print("Done")

Loading Dataset...
Done


## Reshape and Normalize data

In [ ]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_train = x_train.astype('float32')
x_train /= 255
y_train_cat=np.argmax(y_train, 1)
print(y_train_cat)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, stratify=y_train_cat)    

## Define the CNN

In [ ]:
model=Sequential()

#First Convolutional Layer
model.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Dropout(0.15))

#Second Convolutional Layer
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(MaxPooling2D(pool_size=(2,2)))


#Third Convolutional Layer
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.15))

#Fourth Convolutional Layer
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(MaxPooling2D(pool_size=(2,2)))

#Fifth Convolutional Layer
model.add(Conv2D(96, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.15))

#Sixth Convolutional Layer
model.add(Conv2D(96, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(MaxPooling2D(pool_size=(2,2)))

#Seventh Convolutional Layer
model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(Dropout(0.15))

#Eight Convolutional Layer
model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flatten layer
model.add(Flatten())

#First Dense Layer
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))

#Output Layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

#Model summary
print(model.summary())

#Compile the model
opt=optimizers.SGD()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

## Train the model

In [ ]:
stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=4)
history=model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_val, y_val), shuffle=True, callbacks=[stopping])

## Performance metrics plot

In [ ]:
preds= model.predict(x_val)
y_pred = np.argmax(preds, axis=1)
y_val = np.argmax(y_val, axis=1)
cm = confusion_matrix(y_val, y_pred, labels=None, sample_weight=None)
print(cm)
y_pred = keras.utils.to_categorical(y_pred, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
print(classification_report(y_val, y_pred, labels=None, digits=3))

## Confusion matrix Plot

In [ ]:
preds= model.predict(x_val)
y_pred = np.argmax(preds, axis=1)
y_val = np.argmax(y_val, axis=1)


plot_confusion_matrix(y_val, y_pred, classes = np.array(['N', 'A', '~', 'O']), normalize=False)

cm = confusion_matrix(y_val, y_pred)

conf = [] # data structure for confusions: list of (i,j,cm[i][j])
for i in range(0,cm.shape[0]):
    for j in range(0,cm.shape[1]):
        if (i!=j and cm[i][j]>0):
            conf.append([i,j,cm[i][j]])

col=2
conf = np.array(conf)
conf = conf[np.argsort(-conf[:,col])]  # decreasing order by 3-rd column (i.e., cm[i][j])

print(classification_report(y_val, y_pred, labels=None, digits=3))

## Accuracy and Loss Plots

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()